In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

Need to have...
- Separated into L1C prepare vs L2A prepare
- If L1C: checked availability on GCLoud 

In [2]:
import glob
import shutil
from datetime import datetime
import pandas as pd
import time

from utils.prepS2 import *
from utils.yamlUtils import *


In [3]:
non_cogs_dir = "../S2_Download/"
cogs_dir = "../S2_ARD/"
log_file = "./processLog.csv"
sen2cor_8 = "~/Sen2Cor-02.08.00-Linux64/bin/L2A_Process"
sen2cor_5 = "~/Sen2Cor-02.08.00-Linux64/bin/L2A_Process"

s3_bucket = "public-eo-data"
s3_dir = "catapult/sentinel2/"
aws_creds = pd.read_csv("../aws_creds.csv")
access_key = aws_creds.AWSAccessKeyId.values[0]
secret_key = aws_creds.AWSSecretKey.values[0]

In [4]:
des_scenes_list = [
    "S2A_MSIL1C_20180820T223011_N0206_R072_T60KWE_20180821T013410.SAFE",
    "S2A_MSIL1C_20180820T223011_N0206_R072_T60KWF_20180821T013410.SAFE",
    "S2A_MSIL1C_20180820T223011_N0206_R072_T60KXE_20180821T013410.SAFE",
    "S2B_MSIL1C_20190608T221949_N0207_R029_T60KXE_20190608T233246.SAFE",
    "S2B_MSIL1C_20190608T221949_N0207_R029_T60KXF_20190608T233246.SAFE",
    "S2A_MSIL1C_20160930T233812_N0204_R087_T57LWK_20160930T233815.SAFE",
    "S2A_MSIL1C_20160930T233812_N0204_R087_T57LXK_20160930T233815.SAFE",
    "S2B_MSIL1C_20171222T234729_N0206_R130_T57LWK_20171223T005857.SAFE",
    "S2B_MSIL1C_20180725T230019_N0206_R058_T58KHF_20180726T002918.SAFE"
    
]

In [8]:
[find_s2_uuid(x) for x in des_scenes_list]

['da9064f8-5186-408c-8da5-3347d4ae5b76',
 '77da6be6-ddfb-4a64-ad58-085c07a6fb7f',
 '56e18fe8-d091-4540-bcff-5e1bb4f7581a',
 '132890a6-1807-49d7-a7b0-48a3331d0c8d',
 'f95fb5bd-ae2a-4285-aa0c-066a8b92a5f4',
 '39ab4e1d-a52d-47dd-9a98-2c3fa230d0f4',
 '8ae5481e-b9b8-44fb-9cd3-2d09520a4dd9',
 '52713300-23ec-4d6d-af8b-92c1fd7bab87',
 '7b837bd6-4cca-4312-88cc-532fa4b8af76']

In [ ]:
with open(log_file, 'a') as log:

    for des_scene in des_scenes_list:
        log.write("{},{},{}".format(des_scene, 'Starting', str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))))
        log.write("\n")
        # shorten scene name
        scene_name = des_scene[:-21]
        scene_name = scene_name[:-17] + scene_name.split('_')[-1]
        print ( 'Scene name: {}'.format(scene_name) )

        # find uuid for download via esa hub
        s2id = find_s2_uuid(des_scene)

        down_dir = non_cogs_dir + des_scene + '/'
        print ( 'Download scene dir: {}'.format(down_dir) )
        print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )
        download_s2_granule_gcloud(des_scene, non_cogs_dir)
#         download_extract_s2_esa(s2id, non_cogs_dir, down_dir) # using esa due to gcloud metadata issues + sedas confidence

        log.write("{},{},{}".format(des_scene, 'Downloaded', str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))))
        log.write("\n")
        print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )

        # process to l2a
#         l2a_dir = down_dir.replace('_MSIL1C', '_MSIL2A')
#         print ( 'L2A temp dir: {}'.format(l2a_dir) )
#         if not os.path.exists(l2a_dir):
#             sen2cor_correction(sen2cor_8, down_dir, non_cogs_dir) # May need to include try of version 2.5 before certain date - to be determined...

        log.write("{},{},{}".format(des_scene, 'Corrected', str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))))
        log.write("\n")
        print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )

        # convert to cog
        cog_dir = cogs_dir + scene_name.replace('_MSIL1C', '_MSIL2A') + '/'
        print ( 'COG dir: {}'.format(cog_dir) )
        conv_s2scene_cogs(l2a_dir, cog_dir, scene_name.replace('_MSIL1C', '_MSIL2A'))

        log.write("{},{},{}".format(des_scene, 'COGGED', str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))))
        log.write("\n")
        
        copy_s2_metadata(l2a_dir, cog_dir, scene_name.replace('_MSIL1C', '_MSIL2A'))

        create_yaml(cog_dir, 's2')

        log.write("{},{},{}".format(des_scene, 'Yamled', str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))))
        log.write("\n")
        
        print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )



Scene name: S2A_MSIL1C_20180820T223011_T60KWE
Download scene dir: ../S2_Download/S2A_MSIL1C_20180820T223011_N0206_R072_T60KWE_20180821T013410.SAFE/
Time: 2019-08-27 16:31:28
ESA scene already extracted: ../S2_Download/S2A_MSIL1C_20180820T223011_N0206_R072_T60KWE_20180821T013410.SAFE/
Time: 2019-08-27 16:31:28
L2A temp dir: ../S2_Download/S2A_MSIL2A_20180820T223011_N0206_R072_T60KWE_20180821T013410.SAFE/
Time: 2019-08-27 16:31:28
COG dir: ../S2_ARD/S2A_MSIL2A_20180820T223011_T60KWE/
cog already exists: ../S2_ARD/S2A_MSIL2A_20180820T223011_T60KWE/S2A_MSIL2A_20180820T223011_T60KWE_AOT_10m.tif
cog already exists: ../S2_ARD/S2A_MSIL2A_20180820T223011_T60KWE/S2A_MSIL2A_20180820T223011_T60KWE_B02_10m.tif
cog already exists: ../S2_ARD/S2A_MSIL2A_20180820T223011_T60KWE/S2A_MSIL2A_20180820T223011_T60KWE_B03_10m.tif
cog already exists: ../S2_ARD/S2A_MSIL2A_20180820T223011_T60KWE/S2A_MSIL2A_20180820T223011_T60KWE_B04_10m.tif
cog already exists: ../S2_ARD/S2A_MSIL2A_20180820T223011_T60KWE/S2A_MSIL2

Downloading:   3%|▎         | 23.1M/744M [00:25<11:50, 1.01MB/s]

In [ ]:
with open(log_file, 'a') as log:

    for des_scene in des_scenes_list:
        log.write("{},{},{}".format(des_scene, 'Starting', str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))))
        log.write("\n")
        # shorten scene name
        scene_name = des_scene[:-21]
        scene_name = scene_name[:-17] + scene_name.split('_')[-1]
        print ( 'Scene name: {}'.format(scene_name) )

        # find uuid for download via esa hub
        s2id = find_s2_uuid(des_scene)

        down_dir = non_cogs_dir + des_scene + '/'
        print ( 'Download scene dir: {}'.format(down_dir) )
        print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )
        #download_s2_granule_gcloud(des_scene, non_cogs_dir)
        download_extract_s2_esa(s2id, non_cogs_dir, down_dir) # using esa due to gcloud metadata issues + sedas confidence

        log.write("{},{},{}".format(des_scene, 'Downloaded', str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))))
        log.write("\n")
        print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )

        # process to l2a
        l2a_dir = down_dir.replace('_MSIL1C', '_MSIL2A')
        print ( 'L2A temp dir: {}'.format(l2a_dir) )
        if not os.path.exists(l2a_dir):
            sen2cor_correction(sen2cor_8, down_dir, non_cogs_dir) # May need to include try of version 2.5 before certain date - to be determined...

        log.write("{},{},{}".format(des_scene, 'Corrected', str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))))
        log.write("\n")
        print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )

        # convert to cog
        cog_dir = cogs_dir + scene_name.replace('_MSIL1C', '_MSIL2A') + '/'
        print ( 'COG dir: {}'.format(cog_dir) )
        conv_s2scene_cogs(l2a_dir, cog_dir, scene_name.replace('_MSIL1C', '_MSIL2A'))

        log.write("{},{},{}".format(des_scene, 'COGGED', str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))))
        log.write("\n")
        
        copy_s2_metadata(l2a_dir, cog_dir, scene_name.replace('_MSIL1C', '_MSIL2A'))

        create_yaml(cog_dir, 's2')

        log.write("{},{},{}".format(des_scene, 'Yamled', str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))))
        log.write("\n")
        
        print ( 'Time: {}'.format(str(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))) )

